# Utils

In [1]:
def plot_depth_map(dm):
    MIN_DEPTH = 0.0
    MAX_DEPTH = min(1000, np.percentile(dm, 99))

    dm = np.clip(dm, MIN_DEPTH, MAX_DEPTH)
    cmap = plt.cm.plasma_r
    cmap.set_bad(color='black')

    return dm, cmap, MIN_DEPTH, MAX_DEPTH


def resize_keeping_aspect_ratio(img, base, interpolation='area'):
    '''
    Resize the image to a defined length manteining its proportions
    Scaling the shortest side of the image to a fixed 'base' length'
    '''

    if img.shape[0] <= img.shape[1]:
        basewidth = int(base)
        wpercent = (basewidth/float(img.shape[0]))
        hsize = int((float(img.shape[1])*float(wpercent)))
        img = tf.image.resize(img, [basewidth, hsize], method=interpolation, antialias=True)
    else:
        baseheight = int(base)
        wpercent = (baseheight/float(img.shape[1]))
        wsize = int((float(img.shape[0])*float(wpercent)))
        img = tf.image.resize(img, [wsize, baseheight], method=interpolation, antialias=True)

    return np.array(img)
    

def get_zipped_dataset(dataset_path_zipped):
    from zipfile import ZipFile
    with ZipFile(dataset_path_zipped, 'r') as zip: 
        print('Extracting all the files now...') 
        zip.extractall() 
        print('Done!\n')

# Dataset Loading

In [2]:
class DataLoader():
    
    def __init__(self, path):
        self.dataset = path
        self.x = []
        self.y = []
        self.info = 0
    
    def get_test_dataset(self):
        img_path = self.dataset + 'eigen_test_rgb.npy'
        depth_path = self.dataset + 'eigen_test_depth.npy'
        crop = self.dataset + 'eigen_test_crop.npy'

        rgb = np.load(img_path)
        depth = np.load(depth_path)
        crop = np.load(crop)

        self.x = rgb
        self.y = depth

        self.info = rgb.shape[0]

        return self.info


    def load_image(self, W_IMG_reduced_size=192, W_D_reduced_size=48, index=None):    
        if index is None:
            index = np.random.randint(0, self.info)

        # Load RGB Image
        img = self.x[index]

        # Load Depth Image
        depth = np.expand_dims(self.y[index], axis=-1)
            
        # Resize the image to a square image manteining the proportion
        img = resize_keeping_aspect_ratio(img, W_IMG_reduced_size)
        depth = resize_keeping_aspect_ratio(depth, W_D_reduced_size)
        
        return img/255, depth*100, index

# Batch maker

In [3]:
def get_batch_evaluation(batch_size=1, index=None):

    input = np.zeros((batch_size, 192, 256, 3))
    target = np.zeros((batch_size, 48, 64, 1))

    for i in range(batch_size):
        img, depth, _ = dataset_test.load_image(index=index)
        input[i, :, :, :] = np.expand_dims(img, axis=0)
        target[i, :, :, :] = np.expand_dims(depth, axis=0)
    
    return input, target

# Start the Demo

In [ ]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

DATASET_TEST = []
SIZE_TEST = 0
path_weigths = root + '../Models/'
path_dataset_test = root + '../Dataset/nyu_test.zip'

# TFL
!mkdir 'TF_tflite_models/'
tf_ligth_root = '../TF_tflite_models/'
TFLITE_MODEL = tf_ligth_root + 'tf_lite_model.tflite'
    
# Unzip and Load the Dataset
zipped_test = get_zipped_dataset(path_dataset_test)
dataset_test = DataLoader('../')
test_samples = dataset_test.get_test_dataset()
DATASET_TEST.append(dataset_test)
SIZE_TEST += test_samples
print('There are {} images for the Test'.format(test_samples))

In [ ]:
# Load the Model
model = tf.keras.models.load_model(path_weigths + 'model.h5', compile=False)
print('Model loaded\n')

# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

print("TFLite models and their sizes:")
!ls "TF_tflite_models" -lh

In [ ]:
# Load TFLite model and see some details about input/output
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

# Learn about its input and output details
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

#Let's resize input and output tensors, so we can make predictions for batch of images.
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (1, 192, 256, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (1, 48, 64, 1))
tflite_interpreter.allocate_tensors()

# Set input and output details
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

# Compute predictions
in_img, gt_depth = get_batch_evaluation(1, index=532)

tflite_interpreter.set_tensor(input_details[0]['index'], np.array(in_img, dtype=np.float32))

# Infer
t1 = time.time()
tflite_interpreter.invoke()
passed_time = time.time() - t1
print('Elapsed Time = {}\nTime for image: {}'.format(passed_time, (passed_time)))

# Prediction
tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
img_pred, cmap_pred, _, _ = plot_depth_map(tflite_model_predictions)
plt.imshow(tf.squeeze(img_pred), cmap=cmap_pred, vmin=0.0)
cbar = plt.colorbar()
cbar.ax.set_xlabel('cm', size=10, rotation=0)
plt.show()